In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
import lxml
import lxml.etree
import lxml
import lxml.etree
import httplib2
from lxml import html 
from pandas import DataFrame
import re
from IPython.display import FileLink, FileLinks

In [2]:
url_2= 'https://www.drivendata.org/competitions/64/hateful-memes/page/206/'
response_2 = requests.get(url_2)
content = response_2.content
soup = BeautifulSoup(content)
tree = html.fromstring(content)

In [3]:
#Titles for Data driven competation
Title=soup.find_all('h3')
#Price money for competation
Price_Money=soup.find_all("span", attrs={'prize'})

In [4]:
def get_data(pageNo):  
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    r = requests.get('https://www.drivendata.org/competitions/', headers=headers)#, proxies=proxies)
    content = r.content
    soup = BeautifulSoup(content)
    #print(soup)

    alls = []
    for d in soup.findAll('div', attrs={'class':'panel'}):
        title =d.find('h3', attrs={'class':''})
        Reward_amount = d.find('span', attrs={'class':'prize'})
        competation_url = d.find('a', href=True)
        Deliverable=d.find('p')
        Winners=d.find('h4') 
        Judge=d.find('small')
        Dummy=d.find('div', attrs={'class':'user-body'})
        all1=[]
        
        if title is not None:
            #print(price.text)
            all1.append(title.text)
        else:
            all1.append('Title is NA')
   
        if Reward_amount is not None:
            #print(price.text)
            all1.append(Reward_amount.text)
        else:
            all1.append('0')
        if competation_url is not None:
            all1.append(competation_url['href'])
        else:
            all1.append('-')
        if Deliverable is not None:
            all1.append(Deliverable.text)
        else:
            all1.append('NA')
        if Winners is not None:
            all1.append(Winners.text)
        else:
            all1.append('NA')
        if Judge is not None:
            all1.append(Judge.text)
        else:
            all1.append('NA')
        if Dummy is not None:
            all1.append(Dummy.text)
        else:
            all1.append('NA')
        alls.append(all1)    
    return alls

In [5]:
results = []
for i in range(1,2):
    results.append(get_data(i))
flatten = lambda l: [item for sublist in l for item in sublist]
df = pd.DataFrame(flatten(results),columns=['Title','Reward_amount','competation_url','Deliverable','Winners','Judge','Dummy'])
df.to_csv('file_test.csv', index=False, encoding='utf-8')

df = pd.read_csv("file_test.csv")

df['Platform_name'] = 'https://www.drivendata.org'
df.loc[df['Reward_amount'] != '0','Reward_type']= 'Monetary'
df.loc[df['Reward_amount'] == '0','Reward_type']= 'Non_Monetary'
df['competation_url_2'] = df[['Platform_name', 'competation_url']].apply(lambda x: ''.join(x), axis=1)
df['Leader_board']='Yes'
df.drop(['competation_url'],axis=1,inplace=True)
df.loc[df['Judge'] != 'COMPETITION ENDED','COMPETITION_STATUS']= 'OPEN'
df.loc[df['Judge'] == 'COMPETITION ENDED','COMPETITION_STATUS']= 'CLOSED'
df['Group']=df['Dummy'].str.contains('Leaders|1st Place Team')
df.drop(['Dummy'],axis=1,inplace=True)
df.drop(df.head(1).index, inplace=True)

In [7]:
Problem_description = []
for j in range(1,40): 
    url=df.competation_url_2[j]
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    r = requests.get(url, headers=headers)#, proxies=proxies)
    content = r.content
    soup = BeautifulSoup(content)
    tree = html.fromstring(content)
    for i in soup.find_all('div', attrs={'class':'col-md-8 col-md-pull-4'}):
        if i is None:
            Problem_description.append("NA")
        else:
            Problem_description.append(i.text)
df_2 = pd.DataFrame({'Description':Problem_description})

In [8]:
Winner_a = []

Host = []
for j in range(1,40): 
    url=df.competation_url_2[j]
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    r = requests.get(url, headers=headers)#, proxies=proxies)
    content = r.content
    soup = BeautifulSoup(content)
    tree = html.fromstring(content)
    Winner_1 = []
    f = soup.find('tr', attrs={'class':'success'})
    s = soup.find('tr', attrs={'class':'info'})
    t = soup.find('tr', attrs={'class':'warning'})
    Description = soup.find('div', attrs={'class':'col-md-8 col-md-pull-4'})
    J = soup.find('small')
    Winner_b = []
    Winner_c = []
    if soup.find_all('a',{'href': re.compile(r'/competitions/(.*?)')})[2].text.find("Description") == -1:
        if soup.find_all('a',{'href': re.compile(r'/competitions/(.*?)')})[3].text.find("Description") == -1:
            if soup.find_all('a',{'href': re.compile(r'/competitions/(.*?)')})[4].text.find("Description") == -1:
                Winner_b.append('NA')
            else: 
                Winner_b.append('https://www.drivendata.org'+soup.find_all('a',{'href': re.compile(r'/competitions/(.*?)')})[4].attrs['href'])
        else:
            Winner_b.append('https://www.drivendata.org'+soup.find_all('a',{'href': re.compile(r'/competitions/(.*?)')})[3].attrs['href'])           
    else:
        Winner_b.append('https://www.drivendata.org'+soup.find_all('a',{'href': re.compile(r'/competitions/(.*?)')})[2].attrs['href'])
    for i in Winner_b:
        Winner_c.append(i)
       
    if f is not None:
        Winner_1.append(f.text)
    else:
        Winner_1.append(0)
    if s is not None:
        Winner_1.append(s.text)
    else:
        Winner_1.append(0)
    if t is not None:
        Winner_1.append(t.text)
    else:
        Winner_1.append(0)
    if J is not None:
        Winner_1.append(J.text)
    else:
        Winner_1.append('NA')
    if Winner_c is not None:
        Winner_1.append(''.join(map(str,Winner_c)))
    else:
        Winner_1.append('NA')
    Winner_a.append(Winner_1)
    
    
df_3= DataFrame(Winner_a,columns=['First_Winner','Second_Winner','Third_Winner','Host','URL'])   

In [9]:
df_4=pd.concat([df_2,df_3], axis=1, ignore_index=False)
df_4.index = np.arange(1, len(df)+1)
df_f=pd.concat([df,df_4], axis=1, ignore_index=False)
df_f['Deliverable']=df_f['Deliverable'].str.replace('\n'," ")
df_f['Description']=df_f['Description'].str.replace('\n'," ")
df_f['First_Winner']=df_f['First_Winner'].str.replace('\n'," ")
df_f['Second_Winner']=df_f['Second_Winner'].str.replace('\n'," ")
df_f['Third_Winner']=df_f['Third_Winner'].str.replace('\n'," ")

In [10]:
Performance_metric = []
for j in range(1,40):
    metric = []
    if df_f.URL[j] == 'NA':
        metric.append('NA')
    else:   
        url= df_f.URL[j]
        url_2= url
        #print(url_2)
        response_2 = requests.get(url_2)
        content = response_2.content
        soup = BeautifulSoup(content)
        tree = html.fromstring(content)
        for i in tree.xpath('//*[@id="content-page"]/div[*]/div/div[2]/ul/li[*]/a') or tree.xpath('//*[@id="content-page"]/div[*]/div[2]/ul/li[*]/a'):
            if i is None:
                metric.append("NA")
            else:
                metric.append(''.join(map(str,i.text)))                
        
    Performance_metric.append(','.join(map(str,metric)))

In [11]:
Features = []
for j in range(1,40):
    metric = []
    if df_f.URL[j] == 'NA':
        metric.append('NA')
    else:   
        url= df_f.URL[j]
        url_2= url
        response_2 = requests.get(url_2)
        content = response_2.content
        soup = BeautifulSoup(content)
        tree = html.fromstring(content)
        #for i in tree.xpath('//*[@id="content-page"]/div[1]/div/div[1]/ul/li[*]/a'):
                             
        for i in tree.xpath('//*[@id="content-page"]/div[1]/div/div[1]/ul/li[*]/a') or tree.xpath('//*[@id="content-page"]/div[1]/div[1]/ul/li[*]/a'):
            if i is None:
                metric.append("NA")
            else:
                metric.append(''.join(map(str,i.text)))
    Features.append(','.join(map(str,metric)))

In [12]:
Problem_description = []
for j in range(1,40):
    metric = []
    if df_f.URL[j] == 'NA':
        metric.append('NA')
    else:   
        url= df_f.URL[j]
        url_2= url
        response_2 = requests.get(url_2)
        content = response_2.content
        soup = BeautifulSoup(content)
        a = soup.find('div', attrs={'class':'col-md-8 col-md-pull-4'})
        metric.append(a.text)
    Problem_description.append(''.join(map(str,metric)))   

In [13]:
df_f['Data_Type']= Features
df_f['Evaluation_Criteria'] = Performance_metric
df_f['Problem_description'] = Problem_description
df_f['Problem_description']=df_f['Problem_description'].str.replace('\n'," ")
df_f.drop(['Judge'],axis=1,inplace=True)
df_f = df_f.rename(columns={'competation_url_2': 'Competation_URL','First_Winner':'First_Price','Second_Winner':'Second_Winner','Third_Winner':'Third_Price','Deliverable':'Short_Summary'})
df_f.drop(['URL'],axis=1,inplace=True)
df_f

,Title,Reward_amount,Short_Summary,Winners,Platform_name,Reward_type,Competation_URL,Leader_board,COMPETITION_STATUS,Group,Description,First_Price,Second_Winner,Third_Price,Host,Data_Type,Evaluation_Criteria,Problem_description
1,MagNet: Model the Geomagnetic Field,"$30,000",Help NOAA better forecast changes...,Ammarali32,https://www.drivendata.org,Monetary,https://www.drivendata.org/competitions/73/noa...,Yes,OPEN,False,Overview Help NOAA better forecast changes ...,"1st $15,000","2nd $8,000","3rd $5,000",Hosted By NOAA,"List of features,Labels",Root Mean Squared Error,Problem description The goal of this challeng...
2,Differential Privacy Temporal Map Challenge: S...,0,START HERE! Help public safety ag...,rmckenna,https://www.drivendata.org,Non_Monetary,https://www.drivendata.org/competitions/74/com...,Yes,OPEN,False,Overview Large data sets containing persona...,"1st $15,000","2nd $10,000","3rd $5,000",Hosted By NIST PSCR,"Development phase,Final scoring,Overview,Recor...","Calculation,Use,Submissions","Problem description In this contest, your tas..."
3,Differential Privacy Temporal Map Challenge: S...,"$39,000",CALLING PRESCREENED PARTICIPANTS!...,DPSyn_2S,https://www.drivendata.org,Monetary,https://www.drivendata.org/competitions/75/dei...,Yes,OPEN,True,Overview Large data sets containing persona...,"1st $15,000","2nd $10,000","3rd $5,000",Hosted By NIST PSCR,"Development phase,Final scoring,Overview,Recor...","Calculation,Use,Submissions","Problem description In this contest, your tas..."
4,Hateful Memes: Phase 1,0,Detecting hateful content present...,distracted boyfriend,https://www.drivendata.org,Non_Monetary,https://www.drivendata.org/competitions/64/hat...,Yes,OPEN,True,Overview Hateful Memes: Phase 2 is live! In...,"1st $50,000","2nd $25,000","3rd $10,000",Hosted By Facebook,Memes,AUC ROC,Problem description Your goal is to predict w...
5,United Nations Millennium Development Goals,0,The UN's Millennium Development G...,Ganesh221B,https://www.drivendata.org,Non_Monetary,https://www.drivendata.org/competitions/1/unit...,Yes,OPEN,False,"Background In the year 2000, the member stat...",NaN,NaN,NaN,Hosted By DrivenData,,,"Since its founding in 1944, the World Bank ha..."
6,DengAI: Predicting Disease Spread,0,Using environmental data collecte...,cseuom_SolveSoft,https://www.drivendata.org,Non_Monetary,https://www.drivendata.org/competitions/44/den...,Yes,OPEN,True,Challenge Summary Can you predict local epi...,NaN,NaN,NaN,Hosted By DrivenData,"List of features,Example of features",Mean absolute error,Problem description Your goal is to predict t...
7,Pump it Up: Data Mining the Water Table,0,Can you predict which water pumps...,serser,https://www.drivendata.org,Non_Monetary,https://www.drivendata.org/competitions/7/pump...,Yes,OPEN,False,Challenge Summary Can you predict which wat...,NaN,NaN,NaN,Hosted By DrivenData,"List of features,Example of features",List of labels,Problem description This is where you'll find...
8,Richter's Predictor: Modeling Earthquake Damage,0,Can you predict the level of dama...,inoddy,https://www.drivendata.org,Non_Monetary,https://www.drivendata.org/competitions/57/nep...,Yes,OPEN,False,Overview Based on aspects of building locatio...,NaN,NaN,NaN,Hosted By DrivenData,"List of features,Example of features",Example,Problem description We're trying to predict t...
9,Flu Shot Learning: Predict H1N1 and Seasonal F...,0,Can you predict whether people go...,inoddy,https://www.drivendata.org,Non_Monetary,https://www.drivendata.org/competitions/66/flu...,Yes,OPEN,False,Overview Can you predict whether people got ...,NaN,NaN,NaN,Hosted By DrivenData,"Labels,List of features,Example of features","Example,Format example",Problem description Your goal is to predict h...
10,Reboot: Box-Plots for Education,0,We're rebooting our first prized ...,adalseno,https://www.drivendata.org,Non_Monetary,https://www.drivendata.org/competitions/46/box...,Yes,OPEN,False,A New Look at our First Competition We're re...,NaN

In [ ]:
df_f.to_csv('/Users/sampathgubbala/Desktop/export_dataframe.csv', index=True)